# Imports

In [2]:
# %% Imports y configuración
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime
from pathlib import Path
import json

# Agregar el directorio raíz al path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..'))

# Imports del proyecto
from model_ddp.utils.sistem_fun import (
    load_config,
    get_data_path,
    get_artifact_path,
    get_report_path,
    create_experiment_id,
    ensure_directories,
    save_experiment_metadata
)

from model_ddp.simulations.gaussian_simulator import (
    RegressionSimulator,
    SimulationConfig,
    RBFKernel,
    MaternKernel,
    PeriodicKernel,
    TransformationFunctions
)

# Modelos
from model_ddp.models.PSBP_normal_v2 import PSBPNormal

# Metricas y graficas 
from model_ddp.fit.metrics import regression_metrics
from model_ddp.graphics.plots_regression import plot_regression_analysis
from model_ddp.graphics.plots_traces import plot_hyperparameter_traces
from model_ddp.graphics.plots_aplication import plot_credible_intervals

config=load_config()

Los experimentos realizados:
- 1 Feature Linear (n=200) -> 1200 (200 burn)
- 2 Feature Linear (n=200) -> 1200 (200 burn)
- 2 Feature Cuadraticos (n=400) -> 1200 (200 burn)
- 3 Feature Linear, variar Kernel Mater (n=800) ->2400 (400 burn)

Busco evaluar el tiempo de desempeño de la clase LSBP para el caso normal y las mejoras que realice en C++, aqui podra ver como aplicar la mayor parte de las funciones.

In [ ]:
# 1 Feature Linear (n=200) -> 1200 (200 burn)
# 2 Feature Linear (n=200) -> 1200 (200 burn)
# 2 Feature Cuadraticos (n=400) -> 1200 (200 burn)
# 3 Feature Linear, variar Kernel Mater (n=800) ->2400 (400 burn)

# Experimento I 

In [3]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_psbp_001"
SIM_REAL = "simulation"

In [4]:
# Parámetros de ejecución de experimentos 
CARACTERISTICAS = "1 Feature Linear (n=200) -> 1200 (200 burn)"
EXPERIMENT_ID = create_experiment_id("psbp_001_exp_01")

In [5]:
##################################################
# Registrar Experimento
##################################################

# Preparar información del experimento
experiment_data = {
    'experiment_id': EXPERIMENT_ID,
    'nombre': NOMBRE_EJECUCION,
    'tipo': SIM_REAL,
    'descripcion': f"""Experimento: {CARACTERISTICAS}"""
}
registry_file = save_experiment_metadata(config, experiment_data)
print(f"✓ Experimento registrado en: {registry_file}")


✓ Experimento registrado en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\versioning\experiment_registry.md


In [6]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{EXPERIMENT_ID}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=200,
    n_features=1,
    x_range=(0.0, 100.0),
    noise_std=0.2,
    random_state=234
)

# Definir kernel (RBF)
kernel = RBFKernel(
    length_scale=5.0,
    variance=1.0
)


transformation = TransformationFunctions.linear(
    coefficients= [2],
    intercept=[10]
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (200, 1)
  Media por feature: [0.35866669]
  Std por feature: [0.82864792]

Estadísticas de Y:
  Shape: (200,)
  Media: 10.7327
  Std: 1.6713
  Min: 6.4454
  Max: 13.9424
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\psbp_001_exp_01_20251226_205106/_data.csv


### Modelo

In [7]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
psbp_model = PSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=15,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = psbp_model.run(
    iterations=1200,          # Iteraciones totales
    burnin=200               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)


EJECUTANDO LSBPNormal...
Iter 100/1200: K_eff=4, H=102, μ=-0.74, μ₀=9.80, κ₀=0.08, a₀=20.00, b₀=7.85
Iter 200/1200: K_eff=3, H=102, μ=-0.60, μ₀=10.28, κ₀=0.26, a₀=20.00, b₀=8.26
Iter 300/1200: K_eff=3, H=102, μ=-0.63, μ₀=10.19, κ₀=0.15, a₀=20.00, b₀=11.21
Iter 400/1200: K_eff=3, H=102, μ=-0.89, μ₀=10.77, κ₀=0.20, a₀=20.00, b₀=11.00
Iter 500/1200: K_eff=3, H=102, μ=-0.94, μ₀=10.17, κ₀=0.33, a₀=20.00, b₀=9.87
Iter 600/1200: K_eff=3, H=102, μ=-1.02, μ₀=12.18, κ₀=0.26, a₀=20.00, b₀=9.62
Iter 700/1200: K_eff=3, H=102, μ=-0.91, μ₀=15.17, κ₀=0.05, a₀=20.00, b₀=9.83
Iter 800/1200: K_eff=3, H=102, μ=-0.85, μ₀=11.98, κ₀=0.25, a₀=20.00, b₀=9.96
Iter 900/1200: K_eff=3, H=102, μ=-0.68, μ₀=11.10, κ₀=0.32, a₀=20.00, b₀=9.53
Iter 1000/1200: K_eff=3, H=102, μ=-0.77, μ₀=11.22, κ₀=0.79, a₀=20.00, b₀=9.11
Iter 1100/1200: K_eff=3, H=102, μ=-0.95, μ₀=10.98, κ₀=0.38, a₀=20.00, b₀=8.44
Iter 1200/1200: K_eff=3, H=102, μ=-0.75, μ₀=12.05, κ₀=0.29, a₀=20.00, b₀=11.48

LSBP COMPLETADO


### Guardado Modelo

In [8]:
##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################

# Crear carpetas para guardar
artifact_path = get_artifact_path(config, SIM_REAL)
carpeta_modelo = artifact_path / f"{EXPERIMENT_ID}"
carpeta_modelo.mkdir(parents=True, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO MODELO Y RESULTADOS...")
print("="*60)

# 1. Guardar el modelo completo (objeto PSBPNormal)
model_file = carpeta_modelo / "psbp_model.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(psbp_model, f)
print(f"✓ Modelo guardado: {model_file}")

# 2. Guardar solo las trazas (más ligero)
trace_file = carpeta_modelo / "trace.pkl"
with open(trace_file, 'wb') as f:
    pickle.dump(trace, f)
print(f"✓ Trazas guardadas: {trace_file}")

# 3. Guardar resumen posterior
summary = psbp_model.get_posterior_summary()
summary_file = carpeta_modelo / "posterior_summary.json"
with open(summary_file, 'w') as f:
    # Convertir todos los valores a tipos serializables
    summary_json = {}
    for k, v in summary.items():
        if isinstance(v, tuple):
            # Convertir elementos de la tupla
            v0 = v[0].tolist() if hasattr(v[0], 'tolist') else v[0]
            v1 = v[1].tolist() if hasattr(v[1], 'tolist') else v[1]
            summary_json[k] = {'mean': v0, 'std': v1}
        elif hasattr(v, 'tolist'):
            summary_json[k] = v.tolist()
        else:
            summary_json[k] = v
    json.dump(summary_json, f, indent=2)
print(f"✓ Resumen posterior guardado: {summary_file}")

# 4. Guardar metadatos del experimento
metadata = {
    'experiment_id': EXPERIMENT_ID,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'PSBPNormal',
    'data_shape': {
        'n': psbp_model.n,
        'p': psbp_model.p
    },
    'hyperparameters': {
        'H_initial': 15,
        'iterations': 50,
        'burnin': 10,
        'n_grid': psbp_model.n_grid
    },
    'priors': {
        'mu_prior': [float(psbp_model.mu_mu), float(psbp_model.tau_mu_inv)],
        'mu0_prior': [float(psbp_model.m0), float(psbp_model.s02)],
        'kappa0_prior': [float(psbp_model.alpha_kappa), float(psbp_model.beta_kappa)],
        'a0_prior': [float(psbp_model.alpha_a), float(psbp_model.beta_a)],
        'b0_prior': [float(psbp_model.alpha_b), float(psbp_model.beta_b)],
        'psi_prior_mean': psbp_model.mu_psi.tolist() if hasattr(psbp_model.mu_psi, 'tolist') else list(psbp_model.mu_psi),
        'psi_prior_prec': psbp_model.tau_psi.tolist() if hasattr(psbp_model.tau_psi, 'tolist') else list(psbp_model.tau_psi)
    },
    'final_stats': {
        'H_final': psbp_model.H,
        'n_clusters_mean': float(summary['n_clusters'][0]) if isinstance(summary['n_clusters'][0], (np.number, float, int)) else summary['n_clusters'][0],
        'n_clusters_std': float(summary['n_clusters'][1]) if isinstance(summary['n_clusters'][1], (np.number, float, int)) else summary['n_clusters'][1]
    },
    'acceptance_rates': {
        'alpha': float(np.mean(psbp_model.mh_acceptance['alpha'][-100:])) if psbp_model.mh_acceptance['alpha'] else 0.0,
        'psi': float(np.mean(psbp_model.mh_acceptance['psi'][-100:])) if psbp_model.mh_acceptance['psi'] else 0.0,
        'kappa0': float(np.mean(psbp_model.mh_acceptance['kappa0'][-100:])) if psbp_model.mh_acceptance['kappa0'] else 0.0,
        'a0': float(np.mean(psbp_model.mh_acceptance['a0'][-100:])) if psbp_model.mh_acceptance['a0'] else 0.0
    },
    'other_params': {
        'psi_positive': psbp_model.psi_positive,
        'estimate_psi_hyperpriors': psbp_model.estimate_psi_hyperpriors,
        'use_cpp': psbp_model.use_cpp,
        'verbose': psbp_model.verbose
    }
}

metadata_file = carpeta_modelo / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print(f"✓ Metadatos guardados: {metadata_file}")

# 5. Guardar información de normalización 
normalization_file = carpeta_modelo / "normalization.pkl"
normalization_data = {
    'y_mean': float(psbp_model.y_mean) if hasattr(psbp_model.y_mean, '__float__') else psbp_model.y_mean,
    'y_std': float(psbp_model.y_std) if hasattr(psbp_model.y_std, '__float__') else psbp_model.y_std,
    'X_mean': psbp_model.X_mean.tolist() if hasattr(psbp_model.X_mean, 'tolist') else list(psbp_model.X_mean),
    'X_std': psbp_model.X_std.tolist() if hasattr(psbp_model.X_std, 'tolist') else list(psbp_model.X_std)
}
with open(normalization_file, 'wb') as f:
    pickle.dump(normalization_data, f)
print(f"✓ Datos de normalización guardados: {normalization_file}")

print("\n" + "="*60)
print(f"MODELO GUARDADO EN: {carpeta_modelo}")
print("="*60)


GUARDANDO MODELO Y RESULTADOS...
✓ Modelo guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251226_205106\psbp_model.pkl
✓ Trazas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251226_205106\trace.pkl
✓ Resumen posterior guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251226_205106\posterior_summary.json
✓ Metadatos guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251226_205106\metadata.json
✓ Datos de normalización guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251226_205106\normalization.pkl

MODELO GUARDADO EN: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_01_20251226_205106


### Predicciones, grafica de Predicciones y guardado

In [9]:
##################################################
#  FIT  (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO PREDICCIONES...")
print("="*60)

# Hacer predicciones con el modelo entrenado
y_pred_mean, y_pred_std = psbp_model.predict_mean(
    X_new=datos.drop(columns=["Y"]).values,
    n_samples=1000
)

y_true = datos["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})
predictions_file = carpeta_reportes / "predictions.csv"
predictions_df.to_csv(predictions_file, index=False)
print(f"✓ Predicciones guardadas: {predictions_file}")

##################################################
# Gráficas de Fit (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO GRÁFICAS...")
print("="*60)
# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Training Set")
]

plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="PSBP Normal"
)

print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


GENERANDO PREDICCIONES...

📊 MÉTRICAS DE AJUSTE:
------------------------------------------------------------
  MSE     :   0.369674
  RMSE    :   0.608009
  MAE     :   0.463969
  R2      :   0.867656
  MAPE    :   4.780538
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_01_20251226_205106\metrics.json
✓ Predicciones guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_01_20251226_205106\predictions.csv

GENERANDO GRÁFICAS...
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_01_20251226_205106


### Otros analisis

In [10]:
##################################################
# Trazas 
##################################################
# Parametros a ver las trazas
hyperparams = [
    ('mu', 'μ (Intercepto stick-breaking)'),
    ('mu0', 'μ₀ (Media base)'),
    ('kappa0', 'κ₀ (Precisión relativa)'),
    ('a0', 'a₀ (Shape σ²)'),
    ('b0', 'b₀ (Scale σ²)'),    
    ('n_clusters', 'Número de Clusters')   
]

plot_hyperparameter_traces(
    trace=trace,
    param_config=hyperparams,
    output_path=carpeta_graficas / f"{EXPERIMENT_ID}.png",
    title="Trazas PSBP Normal"
)

##################################################
# Gráfica de intervalo predicción media 
##################################################

# Versión simple
plot_credible_intervals(
    y_true=y_true,
    y_pred_mean=y_pred_mean,
    y_pred_std=y_pred_std,
    output_path=carpeta_graficas / "intervalos_credibilidad.png",
    sort_by='y_true'
)

print(f"✓ Todas las gráficas guardadas en: {carpeta_graficas}")

c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:78: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.tight_layout()
c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:83: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.savefig(output_path, dpi=300, bbox_inches='tight')


✓ Todas las gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_01_20251226_205106


# Experimento II 

In [11]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_psbp_001"
SIM_REAL = "simulation"

In [12]:
# Parámetros de ejecución de experimentos 
CARACTERISTICAS = "2 Feature Linear (n=200) -> 1200 (200 burn)"
EXPERIMENT_ID = create_experiment_id("psbp_001_exp_02")

In [13]:
##################################################
# Registrar Experimento
##################################################

# Preparar información del experimento
experiment_data = {
    'experiment_id': EXPERIMENT_ID,
    'nombre': NOMBRE_EJECUCION,
    'tipo': SIM_REAL,
    'descripcion': f"""Experimento: {CARACTERISTICAS}"""
}
registry_file = save_experiment_metadata(config, experiment_data)
print(f"✓ Experimento registrado en: {registry_file}")


✓ Experimento registrado en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\versioning\experiment_registry.md


In [14]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{EXPERIMENT_ID}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=200,
    n_features=2,
    x_range=(0.0, 100.0),
    noise_std=0.2,
    random_state=234
)

# Definir kernel (RBF)
kernel = RBFKernel(
    length_scale=5.0,
    variance=1.0
)


transformation = TransformationFunctions.linear(
    coefficients= [2,15],
    intercept=[10]
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (200, 2)
  Media por feature: [ 0.35866669 -0.30311795]
  Std por feature: [0.82864792 0.75455656]

Estadísticas de Y:
  Shape: (200,)
  Media: 6.1859
  Std: 11.7161
  Min: -17.2658
  Max: 33.8075
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\psbp_001_exp_02_20251226_205410/_data.csv


## Modelo 

In [15]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
psbp_model = PSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=15,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = psbp_model.run(
    iterations=1200,          # Iteraciones totales
    burnin=200               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)


EJECUTANDO LSBPNormal...
Iter 100/1200: K_eff=4, H=102, μ=0.31, μ₀=22.12, κ₀=0.16, a₀=20.00, b₀=613.55
Iter 200/1200: K_eff=3, H=102, μ=0.47, μ₀=11.89, κ₀=0.48, a₀=20.00, b₀=525.23
Iter 300/1200: K_eff=3, H=102, μ=0.40, μ₀=-0.08, κ₀=0.03, a₀=20.00, b₀=353.79
Iter 400/1200: K_eff=3, H=102, μ=0.43, μ₀=6.61, κ₀=0.07, a₀=20.00, b₀=486.67
Iter 500/1200: K_eff=4, H=102, μ=0.53, μ₀=9.84, κ₀=0.12, a₀=20.00, b₀=440.97
Iter 600/1200: K_eff=4, H=102, μ=0.76, μ₀=11.17, κ₀=0.14, a₀=20.00, b₀=368.08
Iter 700/1200: K_eff=5, H=102, μ=0.63, μ₀=1.49, κ₀=0.12, a₀=20.00, b₀=363.44
Iter 800/1200: K_eff=5, H=102, μ=0.49, μ₀=8.00, κ₀=0.13, a₀=20.00, b₀=210.43
Iter 900/1200: K_eff=6, H=102, μ=0.47, μ₀=7.31, κ₀=0.03, a₀=20.00, b₀=148.35
Iter 1000/1200: K_eff=6, H=102, μ=0.46, μ₀=1.94, κ₀=0.09, a₀=20.00, b₀=188.49
Iter 1100/1200: K_eff=6, H=102, μ=0.62, μ₀=4.54, κ₀=0.04, a₀=20.00, b₀=126.01
Iter 1200/1200: K_eff=6, H=102, μ=0.49, μ₀=-0.63, κ₀=0.02, a₀=20.00, b₀=116.28

LSBP COMPLETADO


## Guardado Modelo

In [16]:
##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################

# Crear carpetas para guardar
artifact_path = get_artifact_path(config, SIM_REAL)
carpeta_modelo = artifact_path / f"{EXPERIMENT_ID}"
carpeta_modelo.mkdir(parents=True, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO MODELO Y RESULTADOS...")
print("="*60)

# 1. Guardar el modelo completo (objeto PSBPNormal)
model_file = carpeta_modelo / "psbp_model.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(psbp_model, f)
print(f"✓ Modelo guardado: {model_file}")

# 2. Guardar solo las trazas (más ligero)
trace_file = carpeta_modelo / "trace.pkl"
with open(trace_file, 'wb') as f:
    pickle.dump(trace, f)
print(f"✓ Trazas guardadas: {trace_file}")

# 3. Guardar resumen posterior
summary = psbp_model.get_posterior_summary()
summary_file = carpeta_modelo / "posterior_summary.json"
with open(summary_file, 'w') as f:
    # Convertir todos los valores a tipos serializables
    summary_json = {}
    for k, v in summary.items():
        if isinstance(v, tuple):
            # Convertir elementos de la tupla
            v0 = v[0].tolist() if hasattr(v[0], 'tolist') else v[0]
            v1 = v[1].tolist() if hasattr(v[1], 'tolist') else v[1]
            summary_json[k] = {'mean': v0, 'std': v1}
        elif hasattr(v, 'tolist'):
            summary_json[k] = v.tolist()
        else:
            summary_json[k] = v
    json.dump(summary_json, f, indent=2)
print(f"✓ Resumen posterior guardado: {summary_file}")

# 4. Guardar metadatos del experimento
metadata = {
    'experiment_id': EXPERIMENT_ID,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'PSBPNormal',
    'data_shape': {
        'n': psbp_model.n,
        'p': psbp_model.p
    },
    'hyperparameters': {
        'H_initial': 15,
        'iterations': 50,
        'burnin': 10,
        'n_grid': psbp_model.n_grid
    },
    'priors': {
        'mu_prior': [float(psbp_model.mu_mu), float(psbp_model.tau_mu_inv)],
        'mu0_prior': [float(psbp_model.m0), float(psbp_model.s02)],
        'kappa0_prior': [float(psbp_model.alpha_kappa), float(psbp_model.beta_kappa)],
        'a0_prior': [float(psbp_model.alpha_a), float(psbp_model.beta_a)],
        'b0_prior': [float(psbp_model.alpha_b), float(psbp_model.beta_b)],
        'psi_prior_mean': psbp_model.mu_psi.tolist() if hasattr(psbp_model.mu_psi, 'tolist') else list(psbp_model.mu_psi),
        'psi_prior_prec': psbp_model.tau_psi.tolist() if hasattr(psbp_model.tau_psi, 'tolist') else list(psbp_model.tau_psi)
    },
    'final_stats': {
        'H_final': psbp_model.H,
        'n_clusters_mean': float(summary['n_clusters'][0]) if isinstance(summary['n_clusters'][0], (np.number, float, int)) else summary['n_clusters'][0],
        'n_clusters_std': float(summary['n_clusters'][1]) if isinstance(summary['n_clusters'][1], (np.number, float, int)) else summary['n_clusters'][1]
    },
    'acceptance_rates': {
        'alpha': float(np.mean(psbp_model.mh_acceptance['alpha'][-100:])) if psbp_model.mh_acceptance['alpha'] else 0.0,
        'psi': float(np.mean(psbp_model.mh_acceptance['psi'][-100:])) if psbp_model.mh_acceptance['psi'] else 0.0,
        'kappa0': float(np.mean(psbp_model.mh_acceptance['kappa0'][-100:])) if psbp_model.mh_acceptance['kappa0'] else 0.0,
        'a0': float(np.mean(psbp_model.mh_acceptance['a0'][-100:])) if psbp_model.mh_acceptance['a0'] else 0.0
    },
    'other_params': {
        'psi_positive': psbp_model.psi_positive,
        'estimate_psi_hyperpriors': psbp_model.estimate_psi_hyperpriors,
        'use_cpp': psbp_model.use_cpp,
        'verbose': psbp_model.verbose
    }
}

metadata_file = carpeta_modelo / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print(f"✓ Metadatos guardados: {metadata_file}")

# 5. Guardar información de normalización 
normalization_file = carpeta_modelo / "normalization.pkl"
normalization_data = {
    'y_mean': float(psbp_model.y_mean) if hasattr(psbp_model.y_mean, '__float__') else psbp_model.y_mean,
    'y_std': float(psbp_model.y_std) if hasattr(psbp_model.y_std, '__float__') else psbp_model.y_std,
    'X_mean': psbp_model.X_mean.tolist() if hasattr(psbp_model.X_mean, 'tolist') else list(psbp_model.X_mean),
    'X_std': psbp_model.X_std.tolist() if hasattr(psbp_model.X_std, 'tolist') else list(psbp_model.X_std)
}
with open(normalization_file, 'wb') as f:
    pickle.dump(normalization_data, f)
print(f"✓ Datos de normalización guardados: {normalization_file}")

print("\n" + "="*60)
print(f"MODELO GUARDADO EN: {carpeta_modelo}")
print("="*60)


GUARDANDO MODELO Y RESULTADOS...
✓ Modelo guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251226_205410\psbp_model.pkl
✓ Trazas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251226_205410\trace.pkl
✓ Resumen posterior guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251226_205410\posterior_summary.json
✓ Metadatos guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251226_205410\metadata.json
✓ Datos de normalización guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251226_205410\normalization.pkl

MODELO GUARDADO EN: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_02_20251226_205410


## Pred, graf

In [17]:
##################################################
#  FIT  (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO PREDICCIONES...")
print("="*60)

# Hacer predicciones con el modelo entrenado
y_pred_mean, y_pred_std = psbp_model.predict_mean(
    X_new=datos.drop(columns=["Y"]).values,
    n_samples=1000
)

y_true = datos["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})
predictions_file = carpeta_reportes / "predictions.csv"
predictions_df.to_csv(predictions_file, index=False)
print(f"✓ Predicciones guardadas: {predictions_file}")

##################################################
# Gráficas de Fit (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO GRÁFICAS...")
print("="*60)
# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Training Set")
]

plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="PSBP Normal"
)

print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


GENERANDO PREDICCIONES...

📊 MÉTRICAS DE AJUSTE:
------------------------------------------------------------
  MSE     :  14.387086
  RMSE    :   3.793031
  MAE     :   2.790141
  R2      :   0.895188
  MAPE    :  58.340126
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_02_20251226_205410\metrics.json
✓ Predicciones guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_02_20251226_205410\predictions.csv

GENERANDO GRÁFICAS...
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_02_20251226_205410


## Otros analisis

In [18]:
##################################################
# Trazas 
##################################################
# Parametros a ver las trazas
hyperparams = [
    ('mu', 'μ (Intercepto stick-breaking)'),
    ('mu0', 'μ₀ (Media base)'),
    ('kappa0', 'κ₀ (Precisión relativa)'),
    ('a0', 'a₀ (Shape σ²)'),
    ('b0', 'b₀ (Scale σ²)'),    
    ('n_clusters', 'Número de Clusters')   
]

plot_hyperparameter_traces(
    trace=trace,
    param_config=hyperparams,
    output_path=carpeta_graficas / f"{EXPERIMENT_ID}.png",
    title="Trazas PSBP Normal"
)

##################################################
# Gráfica de intervalo predicción media 
##################################################

# Versión simple
plot_credible_intervals(
    y_true=y_true,
    y_pred_mean=y_pred_mean,
    y_pred_std=y_pred_std,
    output_path=carpeta_graficas / "intervalos_credibilidad.png",
    sort_by='y_true'
)

print(f"✓ Todas las gráficas guardadas en: {carpeta_graficas}")

c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:78: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.tight_layout()
c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:83: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.savefig(output_path, dpi=300, bbox_inches='tight')


✓ Todas las gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_02_20251226_205410


# Experimento III

In [19]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_psbp_001"
SIM_REAL = "simulation"

In [20]:
# Parámetros de ejecución de experimentos 
CARACTERISTICAS = "2 Feature Cuadraticos (n=400) -> 1200 (200 burn)"
EXPERIMENT_ID = create_experiment_id("psbp_001_exp_03")

In [21]:
##################################################
# Registrar Experimento
##################################################

# Preparar información del experimento
experiment_data = {
    'experiment_id': EXPERIMENT_ID,
    'nombre': NOMBRE_EJECUCION,
    'tipo': SIM_REAL,
    'descripcion': f"""Experimento: {CARACTERISTICAS}"""
}
registry_file = save_experiment_metadata(config, experiment_data)
print(f"✓ Experimento registrado en: {registry_file}")


✓ Experimento registrado en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\versioning\experiment_registry.md


In [22]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{EXPERIMENT_ID}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=400,
    n_features=2,
    x_range=(0.0, 100.0),
    noise_std=0.2,
    random_state=234
)

# Definir kernel (RBF)
kernel = RBFKernel(
    length_scale=5.0,
    variance=1.0
)


transformation = TransformationFunctions.polynomial(
    degree=2
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (400, 2)
  Media por feature: [ 0.14187066 -0.50272967]
  Std por feature: [0.78558034 0.87377361]

Estadísticas de Y:
  Shape: (400,)
  Media: 1.6552
  Std: 1.9636
  Min: -0.4204
  Max: 9.8443
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\psbp_001_exp_03_20251226_205802/_data.csv


## Modelo

In [23]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
psbp_model = PSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=15,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = psbp_model.run(
    iterations=1200,          # Iteraciones totales
    burnin=200               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)


EJECUTANDO LSBPNormal...
Iter 100/1200: K_eff=3, H=102, μ=0.22, μ₀=0.62, κ₀=0.22, a₀=20.00, b₀=15.94
Iter 200/1200: K_eff=4, H=102, μ=0.33, μ₀=3.24, κ₀=0.28, a₀=20.00, b₀=13.12
Iter 300/1200: K_eff=3, H=102, μ=0.14, μ₀=2.77, κ₀=0.44, a₀=20.00, b₀=16.59
Iter 400/1200: K_eff=3, H=102, μ=0.19, μ₀=3.19, κ₀=0.15, a₀=20.00, b₀=12.23
Iter 500/1200: K_eff=3, H=102, μ=0.36, μ₀=1.69, κ₀=0.37, a₀=20.00, b₀=9.72
Iter 600/1200: K_eff=3, H=102, μ=0.34, μ₀=1.60, κ₀=0.54, a₀=20.00, b₀=12.16
Iter 700/1200: K_eff=3, H=102, μ=0.41, μ₀=2.06, κ₀=0.82, a₀=20.00, b₀=11.52
Iter 800/1200: K_eff=3, H=102, μ=0.51, μ₀=2.80, κ₀=0.19, a₀=20.00, b₀=15.43
Iter 900/1200: K_eff=5, H=102, μ=0.39, μ₀=2.56, κ₀=0.13, a₀=20.00, b₀=8.64
Iter 1000/1200: K_eff=4, H=102, μ=0.51, μ₀=3.10, κ₀=0.11, a₀=20.00, b₀=12.29
Iter 1100/1200: K_eff=4, H=102, μ=0.40, μ₀=4.01, κ₀=0.07, a₀=20.00, b₀=12.86
Iter 1200/1200: K_eff=4, H=102, μ=0.52, μ₀=5.35, κ₀=0.06, a₀=20.00, b₀=13.45

LSBP COMPLETADO


## Guardado Modelo

In [24]:
##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################

# Crear carpetas para guardar
artifact_path = get_artifact_path(config, SIM_REAL)
carpeta_modelo = artifact_path / f"{EXPERIMENT_ID}"
carpeta_modelo.mkdir(parents=True, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO MODELO Y RESULTADOS...")
print("="*60)

# 1. Guardar el modelo completo (objeto PSBPNormal)
model_file = carpeta_modelo / "psbp_model.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(psbp_model, f)
print(f"✓ Modelo guardado: {model_file}")

# 2. Guardar solo las trazas (más ligero)
trace_file = carpeta_modelo / "trace.pkl"
with open(trace_file, 'wb') as f:
    pickle.dump(trace, f)
print(f"✓ Trazas guardadas: {trace_file}")

# 3. Guardar resumen posterior
summary = psbp_model.get_posterior_summary()
summary_file = carpeta_modelo / "posterior_summary.json"
with open(summary_file, 'w') as f:
    # Convertir todos los valores a tipos serializables
    summary_json = {}
    for k, v in summary.items():
        if isinstance(v, tuple):
            # Convertir elementos de la tupla
            v0 = v[0].tolist() if hasattr(v[0], 'tolist') else v[0]
            v1 = v[1].tolist() if hasattr(v[1], 'tolist') else v[1]
            summary_json[k] = {'mean': v0, 'std': v1}
        elif hasattr(v, 'tolist'):
            summary_json[k] = v.tolist()
        else:
            summary_json[k] = v
    json.dump(summary_json, f, indent=2)
print(f"✓ Resumen posterior guardado: {summary_file}")

# 4. Guardar metadatos del experimento
metadata = {
    'experiment_id': EXPERIMENT_ID,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'PSBPNormal',
    'data_shape': {
        'n': psbp_model.n,
        'p': psbp_model.p
    },
    'hyperparameters': {
        'H_initial': 15,
        'iterations': 50,
        'burnin': 10,
        'n_grid': psbp_model.n_grid
    },
    'priors': {
        'mu_prior': [float(psbp_model.mu_mu), float(psbp_model.tau_mu_inv)],
        'mu0_prior': [float(psbp_model.m0), float(psbp_model.s02)],
        'kappa0_prior': [float(psbp_model.alpha_kappa), float(psbp_model.beta_kappa)],
        'a0_prior': [float(psbp_model.alpha_a), float(psbp_model.beta_a)],
        'b0_prior': [float(psbp_model.alpha_b), float(psbp_model.beta_b)],
        'psi_prior_mean': psbp_model.mu_psi.tolist() if hasattr(psbp_model.mu_psi, 'tolist') else list(psbp_model.mu_psi),
        'psi_prior_prec': psbp_model.tau_psi.tolist() if hasattr(psbp_model.tau_psi, 'tolist') else list(psbp_model.tau_psi)
    },
    'final_stats': {
        'H_final': psbp_model.H,
        'n_clusters_mean': float(summary['n_clusters'][0]) if isinstance(summary['n_clusters'][0], (np.number, float, int)) else summary['n_clusters'][0],
        'n_clusters_std': float(summary['n_clusters'][1]) if isinstance(summary['n_clusters'][1], (np.number, float, int)) else summary['n_clusters'][1]
    },
    'acceptance_rates': {
        'alpha': float(np.mean(psbp_model.mh_acceptance['alpha'][-100:])) if psbp_model.mh_acceptance['alpha'] else 0.0,
        'psi': float(np.mean(psbp_model.mh_acceptance['psi'][-100:])) if psbp_model.mh_acceptance['psi'] else 0.0,
        'kappa0': float(np.mean(psbp_model.mh_acceptance['kappa0'][-100:])) if psbp_model.mh_acceptance['kappa0'] else 0.0,
        'a0': float(np.mean(psbp_model.mh_acceptance['a0'][-100:])) if psbp_model.mh_acceptance['a0'] else 0.0
    },
    'other_params': {
        'psi_positive': psbp_model.psi_positive,
        'estimate_psi_hyperpriors': psbp_model.estimate_psi_hyperpriors,
        'use_cpp': psbp_model.use_cpp,
        'verbose': psbp_model.verbose
    }
}

metadata_file = carpeta_modelo / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print(f"✓ Metadatos guardados: {metadata_file}")

# 5. Guardar información de normalización 
normalization_file = carpeta_modelo / "normalization.pkl"
normalization_data = {
    'y_mean': float(psbp_model.y_mean) if hasattr(psbp_model.y_mean, '__float__') else psbp_model.y_mean,
    'y_std': float(psbp_model.y_std) if hasattr(psbp_model.y_std, '__float__') else psbp_model.y_std,
    'X_mean': psbp_model.X_mean.tolist() if hasattr(psbp_model.X_mean, 'tolist') else list(psbp_model.X_mean),
    'X_std': psbp_model.X_std.tolist() if hasattr(psbp_model.X_std, 'tolist') else list(psbp_model.X_std)
}
with open(normalization_file, 'wb') as f:
    pickle.dump(normalization_data, f)
print(f"✓ Datos de normalización guardados: {normalization_file}")

print("\n" + "="*60)
print(f"MODELO GUARDADO EN: {carpeta_modelo}")
print("="*60)


GUARDANDO MODELO Y RESULTADOS...
✓ Modelo guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251226_205802\psbp_model.pkl
✓ Trazas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251226_205802\trace.pkl
✓ Resumen posterior guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251226_205802\posterior_summary.json
✓ Metadatos guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251226_205802\metadata.json
✓ Datos de normalización guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251226_205802\normalization.pkl

MODELO GUARDADO EN: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_03_20251226_205802


## pred Graf

In [25]:
##################################################
#  FIT  (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO PREDICCIONES...")
print("="*60)

# Hacer predicciones con el modelo entrenado
y_pred_mean, y_pred_std = psbp_model.predict_mean(
    X_new=datos.drop(columns=["Y"]).values,
    n_samples=1000
)

y_true = datos["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})
predictions_file = carpeta_reportes / "predictions.csv"
predictions_df.to_csv(predictions_file, index=False)
print(f"✓ Predicciones guardadas: {predictions_file}")

##################################################
# Gráficas de Fit (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO GRÁFICAS...")
print("="*60)
# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Training Set")
]

plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="PSBP Normal"
)

print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


GENERANDO PREDICCIONES...

📊 MÉTRICAS DE AJUSTE:
------------------------------------------------------------
  MSE     :   0.446141
  RMSE    :   0.667938
  MAE     :   0.463672
  R2      :   0.884297
  MAPE    : 157.909477
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_03_20251226_205802\metrics.json
✓ Predicciones guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_03_20251226_205802\predictions.csv

GENERANDO GRÁFICAS...
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_03_20251226_205802


## Otros 

In [26]:
##################################################
# Trazas 
##################################################
# Parametros a ver las trazas
hyperparams = [
    ('mu', 'μ (Intercepto stick-breaking)'),
    ('mu0', 'μ₀ (Media base)'),
    ('kappa0', 'κ₀ (Precisión relativa)'),
    ('a0', 'a₀ (Shape σ²)'),
    ('b0', 'b₀ (Scale σ²)'),    
    ('n_clusters', 'Número de Clusters')   
]

plot_hyperparameter_traces(
    trace=trace,
    param_config=hyperparams,
    output_path=carpeta_graficas / f"{EXPERIMENT_ID}.png",
    title="Trazas PSBP Normal"
)

##################################################
# Gráfica de intervalo predicción media 
##################################################

# Versión simple
plot_credible_intervals(
    y_true=y_true,
    y_pred_mean=y_pred_mean,
    y_pred_std=y_pred_std,
    output_path=carpeta_graficas / "intervalos_credibilidad.png",
    sort_by='y_true'
)

print(f"✓ Todas las gráficas guardadas en: {carpeta_graficas}")

c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:78: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.tight_layout()
c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:83: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.savefig(output_path, dpi=300, bbox_inches='tight')


✓ Todas las gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_03_20251226_205802


# Experimento IV

In [27]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_psbp_001"
SIM_REAL = "simulation"

In [28]:
# Parámetros de ejecución de experimentos 
CARACTERISTICAS = "3 Feature Linear, variar Kernel Mater (n=800) ->2400 (400 burn)"
EXPERIMENT_ID = create_experiment_id("psbp_001_exp_04")

In [29]:
##################################################
# Registrar Experimento
##################################################

# Preparar información del experimento
experiment_data = {
    'experiment_id': EXPERIMENT_ID,
    'nombre': NOMBRE_EJECUCION,
    'tipo': SIM_REAL,
    'descripcion': f"""Experimento: {CARACTERISTICAS}"""
}
registry_file = save_experiment_metadata(config, experiment_data)
print(f"✓ Experimento registrado en: {registry_file}")


✓ Experimento registrado en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\versioning\experiment_registry.md


In [30]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{EXPERIMENT_ID}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=800,
    n_features=3,
    x_range=(0.0, 100.0),
    noise_std=0.2,
    random_state=234
)

# Definir kernel (RBF)
kernel = MaternKernel(
    length_scale=5.0,
    variance=2.0
)


transformation = TransformationFunctions.linear(
    coefficients=[15,-5,3],
    intercept=[100]
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (800, 3)
  Media por feature: [-0.23289032  0.35779777  0.13681772]
  Std por feature: [1.11561901 0.93767766 1.07967819]

Estadísticas de Y:
  Shape: (800,)
  Media: 95.1227
  Std: 16.0322
  Min: 65.3149
  Max: 136.1139
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\psbp_001_exp_04_20251226_210318/_data.csv


## Modelo

In [31]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
psbp_model = PSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=30,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = psbp_model.run(
    iterations=2400,          # Iteraciones totales
    burnin=400               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)


EJECUTANDO LSBPNormal...
Iter 100/2400: K_eff=5, H=102, μ=-0.82, μ₀=115.38, κ₀=0.47, a₀=20.00, b₀=1244.72
Iter 200/2400: K_eff=3, H=102, μ=-0.71, μ₀=103.45, κ₀=0.54, a₀=20.00, b₀=872.13
Iter 300/2400: K_eff=3, H=102, μ=-0.53, μ₀=101.77, κ₀=0.35, a₀=20.00, b₀=1351.76
Iter 400/2400: K_eff=3, H=102, μ=-0.57, μ₀=108.45, κ₀=0.57, a₀=20.00, b₀=837.10
Iter 500/2400: K_eff=4, H=102, μ=-0.60, μ₀=110.77, κ₀=0.19, a₀=20.00, b₀=717.06
Iter 600/2400: K_eff=5, H=102, μ=-0.65, μ₀=106.52, κ₀=0.33, a₀=20.00, b₀=814.91
Iter 700/2400: K_eff=5, H=102, μ=-0.68, μ₀=107.40, κ₀=0.40, a₀=20.00, b₀=760.22
Iter 800/2400: K_eff=4, H=102, μ=-0.53, μ₀=119.25, κ₀=0.25, a₀=20.00, b₀=773.01
Iter 900/2400: K_eff=4, H=102, μ=-0.58, μ₀=98.56, κ₀=0.10, a₀=20.00, b₀=856.87
Iter 1000/2400: K_eff=4, H=102, μ=-0.49, μ₀=110.90, κ₀=0.25, a₀=20.00, b₀=706.47
Iter 1100/2400: K_eff=4, H=102, μ=-0.43, μ₀=114.05, κ₀=0.23, a₀=20.00, b₀=848.52
Iter 1200/2400: K_eff=4, H=102, μ=-0.60, μ₀=104.43, κ₀=0.19, a₀=20.00, b₀=749.35
Iter 1300/

## Guardado Modelo

In [34]:
##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################

# Crear carpetas para guardar
artifact_path = get_artifact_path(config, SIM_REAL)
carpeta_modelo = artifact_path / f"{EXPERIMENT_ID}"
carpeta_modelo.mkdir(parents=True, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO MODELO Y RESULTADOS...")
print("="*60)

# 1. Guardar el modelo completo (objeto PSBPNormal)
model_file = carpeta_modelo / "psbp_model.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(psbp_model, f)
print(f"✓ Modelo guardado: {model_file}")

# 2. Guardar solo las trazas (más ligero)
trace_file = carpeta_modelo / "trace.pkl"
with open(trace_file, 'wb') as f:
    pickle.dump(trace, f)
print(f"✓ Trazas guardadas: {trace_file}")

# 3. Guardar resumen posterior
summary = psbp_model.get_posterior_summary()
summary_file = carpeta_modelo / "posterior_summary.json"
with open(summary_file, 'w') as f:
    # Convertir todos los valores a tipos serializables
    summary_json = {}
    for k, v in summary.items():
        if isinstance(v, tuple):
            # Convertir elementos de la tupla
            v0 = v[0].tolist() if hasattr(v[0], 'tolist') else v[0]
            v1 = v[1].tolist() if hasattr(v[1], 'tolist') else v[1]
            summary_json[k] = {'mean': v0, 'std': v1}
        elif hasattr(v, 'tolist'):
            summary_json[k] = v.tolist()
        else:
            summary_json[k] = v
    json.dump(summary_json, f, indent=2)
print(f"✓ Resumen posterior guardado: {summary_file}")

# 4. Guardar metadatos del experimento
metadata = {
    'experiment_id': EXPERIMENT_ID,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'PSBPNormal',
    'data_shape': {
        'n': psbp_model.n,
        'p': psbp_model.p
    },
    'hyperparameters': {
        'H_initial': 15,
        'iterations': 50,
        'burnin': 10,
        'n_grid': psbp_model.n_grid
    },
    'priors': {
        'mu_prior': [float(psbp_model.mu_mu), float(psbp_model.tau_mu_inv)],
        'mu0_prior': [float(psbp_model.m0), float(psbp_model.s02)],
        'kappa0_prior': [float(psbp_model.alpha_kappa), float(psbp_model.beta_kappa)],
        'a0_prior': [float(psbp_model.alpha_a), float(psbp_model.beta_a)],
        'b0_prior': [float(psbp_model.alpha_b), float(psbp_model.beta_b)],
        'psi_prior_mean': psbp_model.mu_psi.tolist() if hasattr(psbp_model.mu_psi, 'tolist') else list(psbp_model.mu_psi),
        'psi_prior_prec': psbp_model.tau_psi.tolist() if hasattr(psbp_model.tau_psi, 'tolist') else list(psbp_model.tau_psi)
    },
    'final_stats': {
        'H_final': psbp_model.H,
        'n_clusters_mean': float(summary['n_clusters'][0]) if isinstance(summary['n_clusters'][0], (np.number, float, int)) else summary['n_clusters'][0],
        'n_clusters_std': float(summary['n_clusters'][1]) if isinstance(summary['n_clusters'][1], (np.number, float, int)) else summary['n_clusters'][1]
    },
    'acceptance_rates': {
        'alpha': float(np.mean(psbp_model.mh_acceptance['alpha'][-100:])) if psbp_model.mh_acceptance['alpha'] else 0.0,
        'psi': float(np.mean(psbp_model.mh_acceptance['psi'][-100:])) if psbp_model.mh_acceptance['psi'] else 0.0,
        'kappa0': float(np.mean(psbp_model.mh_acceptance['kappa0'][-100:])) if psbp_model.mh_acceptance['kappa0'] else 0.0,
        'a0': float(np.mean(psbp_model.mh_acceptance['a0'][-100:])) if psbp_model.mh_acceptance['a0'] else 0.0
    },
    'other_params': {
        'psi_positive': psbp_model.psi_positive,
        'estimate_psi_hyperpriors': psbp_model.estimate_psi_hyperpriors,
        'use_cpp': psbp_model.use_cpp,
        'verbose': psbp_model.verbose
    }
}

metadata_file = carpeta_modelo / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print(f"✓ Metadatos guardados: {metadata_file}")

# 5. Guardar información de normalización 
normalization_file = carpeta_modelo / "normalization.pkl"
normalization_data = {
    'y_mean': float(psbp_model.y_mean) if hasattr(psbp_model.y_mean, '__float__') else psbp_model.y_mean,
    'y_std': float(psbp_model.y_std) if hasattr(psbp_model.y_std, '__float__') else psbp_model.y_std,
    'X_mean': psbp_model.X_mean.tolist() if hasattr(psbp_model.X_mean, 'tolist') else list(psbp_model.X_mean),
    'X_std': psbp_model.X_std.tolist() if hasattr(psbp_model.X_std, 'tolist') else list(psbp_model.X_std)
}
with open(normalization_file, 'wb') as f:
    pickle.dump(normalization_data, f)
print(f"✓ Datos de normalización guardados: {normalization_file}")

print("\n" + "="*60)
print(f"MODELO GUARDADO EN: {carpeta_modelo}")
print("="*60)


GUARDANDO MODELO Y RESULTADOS...
✓ Modelo guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318\psbp_model.pkl
✓ Trazas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318\trace.pkl
✓ Resumen posterior guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318\posterior_summary.json
✓ Metadatos guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318\metadata.json
✓ Datos de normalización guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318\normalization.pkl

MODELO GUARDADO EN: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\psbp_001_exp_04_20251226_210318


## Pred, graf

In [32]:
##################################################
#  FIT  (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO PREDICCIONES...")
print("="*60)

# Hacer predicciones con el modelo entrenado
y_pred_mean, y_pred_std = psbp_model.predict_mean(
    X_new=datos.drop(columns=["Y"]).values,
    n_samples=1000
)

y_true = datos["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})
predictions_file = carpeta_reportes / "predictions.csv"
predictions_df.to_csv(predictions_file, index=False)
print(f"✓ Predicciones guardadas: {predictions_file}")

##################################################
# Gráficas de Fit (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO GRÁFICAS...")
print("="*60)
# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Training Set")
]

plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="PSBP Normal"
)

print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


GENERANDO PREDICCIONES...

📊 MÉTRICAS DE AJUSTE:
------------------------------------------------------------
  MSE     :  43.983346
  RMSE    :   6.631994
  MAE     :   4.890547
  R2      :   0.828879
  MAPE    :   5.492428
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_04_20251226_210318\metrics.json
✓ Predicciones guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_04_20251226_210318\predictions.csv

GENERANDO GRÁFICAS...
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_04_20251226_210318


## Otros

In [33]:
##################################################
# Trazas 
##################################################
# Parametros a ver las trazas
hyperparams = [
    ('mu', 'μ (Intercepto stick-breaking)'),
    ('mu0', 'μ₀ (Media base)'),
    ('kappa0', 'κ₀ (Precisión relativa)'),
    ('a0', 'a₀ (Shape σ²)'),
    ('b0', 'b₀ (Scale σ²)'),    
    ('n_clusters', 'Número de Clusters')   
]

plot_hyperparameter_traces(
    trace=trace,
    param_config=hyperparams,
    output_path=carpeta_graficas / f"{EXPERIMENT_ID}.png",
    title="Trazas PSBP Normal"
)

##################################################
# Gráfica de intervalo predicción media 
##################################################

# Versión simple
plot_credible_intervals(
    y_true=y_true,
    y_pred_mean=y_pred_mean,
    y_pred_std=y_pred_std,
    output_path=carpeta_graficas / "intervalos_credibilidad.png",
    sort_by='y_true'
)

print(f"✓ Todas las gráficas guardadas en: {carpeta_graficas}")

c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:78: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.tight_layout()
c:\Users\JuanFran\Desktop\git_tesis\model_ddp\notebooks\..\model_ddp\graphics\plots_traces.py:83: UserWarning: Glyph 8320 (\N{SUBSCRIPT ZERO}) missing from font(s) Arial.
  plt.savefig(output_path, dpi=300, bbox_inches='tight')


✓ Todas las gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\psbp_001_exp_04_20251226_210318
